In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import collections
from tqdm.notebook import  tqdm
import matplotlib.ticker as ticker
import json

# DATA

## EP_Cit_Counts

In [ ]:
EP_Cit_Counts=pd.read_table('201609_EP_Cit_Counts.txt',sep='|')

In [ ]:
list_EP_Cit_Counts=list(map(list,zip(list(EP_Cit_Counts['EP_Pub_date']),list(EP_Cit_Counts['EP_Pub_nbr']))))
All_Dates=[]
for i in range(len(list_EP_Cit_Counts)):
    All_Dates.append([str(list_EP_Cit_Counts[i][0])[0:4],str(list_EP_Cit_Counts[i][0])[4:6],\
                      str(list_EP_Cit_Counts[i][0])[6:8],list_EP_Cit_Counts[i][1],str(list_EP_Cit_Counts[i][0])])

list_EP_Cit_Counts=All_Dates #2D List με ημερομηνίες σπασμένες και EP_Pub_nbr

In [ ]:
indexes=[] # Πετάω έξω όλες τις σειρές με ημερομηνία '99991231'
for i in range (len(list_EP_Cit_Counts)):
    if list_EP_Cit_Counts[i][0]=='9999' or len(list_EP_Cit_Counts[i][3])!=9:
        indexes.append(i)
indexes.sort(reverse=True) #Αντιστρέφω τα indexes για να μπορώ να διαγράφω τιμές χωρίς να βγαίνει εκτός εύρους
for i in indexes:
    del list_EP_Cit_Counts[i]

# Πετάω έξω διπλότυπα Pub_nbr και sortάρω
test_list_EP_Cit_Counts=[list(t) for t in set(tuple(element) for element in list_EP_Cit_Counts)]
list_EP_Cit_Counts=sorted(test_list_EP_Cit_Counts,key=lambda l:l[3]) #sta alla gt thelei sort me date? pws leitourgei sta tw?

## EPO_App_reg

In [ ]:
EPO_App_reg=pd.read_table('201602_EPO_App_reg.txt',sep='|')
EPO_App_reg=EPO_App_reg.sort_values(by=['Pub_nbr'],ignore_index=True)

# EPO_Ipc

In [ ]:
EPO_Ipc=pd.read_table('201602_EPO_IPC.txt',sep='|')
EPO_Ipc=EPO_Ipc.sort_values(by=['Appln_id'],ignore_index=True)

In [ ]:
list_Ipc=list(map(list,zip(list(EPO_Ipc['Appln_id']),list(EPO_Ipc['IPC']))))
list_Ipc_A=[c for c in list_Ipc if str(c[1])[0]=='A']
list_Ipc_B=[c for c in list_Ipc if str(c[1])[0]=='B']
list_Ipc_C=[c for c in list_Ipc if str(c[1])[0]=='C']
#list_Ipc_D=[c for c in list_Ipc if str(c[1])[0]=='D']
#list_Ipc_E=[c for c in list_Ipc if str(c[1])[0]=='E']
#list_Ipc_F=[c for c in list_Ipc if str(c[1])[0]=='F']
list_Ipc_G=[c for c in list_Ipc if str(c[1])[0]=='G']
list_Ipc_H=[c for c in list_Ipc if str(c[1])[0]=='H']

## EPO_Inv_reg

In [ ]:
EPO_Inv_reg=pd.read_table('201602_EPO_Inv_reg.txt',sep='|')
EPO_Inv_reg=EPO_Inv_reg.sort_values(by=['Pub_nbr'],ignore_index=True)

# FUNCTIONS ! ! !

## Functions to take data

In [ ]:
def takeZeroColumn(elem): # Συνάρτηση για να παίρνει συγκεκριμένη στήλη από 2D list
    return elem[0]

def takeSecondColumn(elem): # Συνάρτηση για να παίρνει συγκεκριμένη στήλη από 2D list ως key για σορτάρισμα
    return elem[2]

def takeThirdColumn(elem):
    return elem[3]

In [ ]:
# Συνάρτηση για να σκανάρω τις δύο λίστες(τις λίστες με τα App/Inv και με τις ημερομηνίες που έχουν Pub_nbr)
# και να κρατήσω μόνο τις σειρές με τα pub_nbrs που υπάρχουν και στις δύο λίστες
# (εδώ ελεγχει λίγες και παίρνει τις ίδιες)
def pub_nbr_person_id_fun(data,tw_data):
    output=[]
    k,l=0,0
    for i in range(k,len(tw_data)):
        for j in range(l,len(data)):
            if tw_data[i][3]==data[j][0]:
                output.append([data[j][0],data[j][1],data[j][2],tw_data[i][4]])
                l=j+1
            elif tw_data[i][3]<data[j][0]:
                break
            elif tw_data[i][3]>data[j][0]:
                pass
    output.sort(key=takeThirdColumn) #sort by date gia na kanw tis syndeseis
    return output

## Functions for NETWORK and creating edges for all data

In [ ]:
def component_dict_fun(data):
    clusters_dict=collections.defaultdict(list)
    clusters=[c for c in sorted(nx.connected_components(data), key=len, reverse=True)]
    for i,c in enumerate(clusters):#Loop through the list of communities,keeping track of the number for the community
        for name in c: # Loop through each person in a community
            clusters_dict[i].append(name) # Create an entry in the dictionary for the person,
                                                   # where the value is which group they belong to.
    # Now you can add modularity information like we did the other metrics
    nx.set_node_attributes(data, clusters_dict, 'components')
    return clusters_dict

In [ ]:
def creating_edges_fun_dynamical(data,step):
    links,clusters,size,net_edges,date=[],[],[],[],[]
    network=nx.Graph()
    k=0
    for i in tqdm(range(len(data)-1), desc='progress_out'):
        for j in range(i+1,len(data)):
            if data[i][0]==data[j][0] and data[i][1]!=data[j][1]: # Ίδια Pub_nbrs αλλά διαφορετικά Person_id
                # ΠΡΟΣΘΗΚΗ!!!!! Δεν κάνει συνδέσεις κόμβων με τον εαυτό τους!!!!
                links.append((data[i][1],data[j][1]))
                k=k+1
                if k%step==0: #or j==len(data): #Na to tsekarw
                    network.add_edges_from(links)
                    clusters=component_dict_fun(network)
                    size.append([len(clusters[0]),len(clusters[1]),len(clusters[2]),\
                                len(clusters[3]),len(clusters[4])])
                    net_edges.append(network.number_of_edges())
                    links.clear()
                    date.append(str(data[i][3][0:4])+'/'+str(data[i][3][4:6])+'/'+str(data[i][3][6:8]))
            else:
                break # Είναι sortarismeno το EPO_App_reg_Pub_nbr_Person_id οπότε κόβω το scannarisma στο 1ο ζευγάρι που δεν είναι ίδιο.
    return net_edges, size, date

## Function for data based on patents' category

In [ ]:
def appln_id_pub_nbr_person_id_fun(data,ipc_data):
    output=[]
    k,l=0,0
    for i in range(k,len(ipc_data)):
        for j in range(l,len(data)):
            if ipc_data[i][0]==data[j][2]:
                output.append([data[j][0],data[j][1],data[j][2],data[j][3]])
                l=j+1
            elif ipc_data[i][0]<data[j][2]:
                break
            elif ipc_data[i][0]>data[j][2]:
                pass
    output.sort(key=takeThirdColumn) #sort by date gia na kanw tis syndeseis
    return output

# EPO_App_reg

## Pub_nbrs

In [ ]:
EPO_App_reg_all=[]
EPO_App_reg_all=list(map(list,zip(EPO_App_reg['Pub_nbr'],EPO_App_reg['Person_id'],EPO_App_reg['Appln_id'])))
#EPO_App_reg_all.sort(key=takeZeroColumn) # Έχω sortάρει στο αρχικό αρχείο

In [ ]:
# Τελικές τιμές Pub_nbrs, Person_ids μετά την αντιστοίχιση στις 2 λίστες.
EPO_App_reg_Pub_nbr_Person_id=[]
EPO_App_reg_Pub_nbr_Person_id=pub_nbr_person_id_fun(EPO_App_reg_all,list_EP_Cit_Counts)

## Creating Edges

In [ ]:
mixed_edges_clusters_EPO_App_reg=[]
links_EPO_App_reg,sizes_EPO_App_reg,dates_EPO_App_reg=[],[],[]
mixed_edges_clusters_EPO_App_reg=creating_edges_fun_dynamical(EPO_App_reg_Pub_nbr_Person_id,500)
links_EPO_App_reg=mixed_edges_clusters_EPO_App_reg[0]
sizes_EPO_App_reg=mixed_edges_clusters_EPO_App_reg[1]
dates_EPO_App_reg=mixed_edges_clusters_EPO_App_reg[2]

In [ ]:
for i in range (len(links_EPO_App_reg)):
    plt.plot(links_EPO_App_reg[i],sizes_EPO_App_reg[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_App_reg[i],sizes_EPO_App_reg[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_App_reg[i],sizes_EPO_App_reg[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_App_reg[i],sizes_EPO_App_reg[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_App_reg[i],sizes_EPO_App_reg[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_App_reg - Full Network",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_App_reg_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_App_reg)):
    EPO_App_reg_DataPlot[i].append([links_EPO_App_reg[i],\
                                    sizes_EPO_App_reg[i][0],\
                                    sizes_EPO_App_reg[i][1],\
                                    sizes_EPO_App_reg[i][2],\
                                    sizes_EPO_App_reg[i][3],\
                                    sizes_EPO_App_reg[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_App_reg__DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_App_reg_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_App_reg_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_App_reg)):
    EPO_App_reg_Data[i].append([dates_EPO_App_reg[i],sizes_EPO_App_reg[i]])

with open('FULL_EPO_App_reg__Data.txt', 'w') as convert_file:
     for key,value in EPO_App_reg_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

# EPO_App_reg based on pantents' category

In [ ]:
EPO_App_reg_Appln_id=[] #Παίρνω το προηγούμενο αρχείο και σορτάρω με βάση το appln_id για αντιστοίχιση παρακάτω
EPO_App_reg_Appln_id=EPO_App_reg_Pub_nbr_Person_id
EPO_App_reg_Appln_id.sort(key=takeSecondColumn)

## Patents "C": Chemistry; Metallurgy

In [ ]:
EPO_App_reg_C_Pub_nbr_Person_id_Appl_id=[] #Aντιστοίχιση για να κρατήσω μόνο τις σειρές με appln_id -> C
EPO_App_reg_C_Pub_nbr_Person_id_Appl_id=appln_id_pub_nbr_person_id_fun(EPO_App_reg_Appln_id,list_Ipc_C)

### Creating Edges

In [ ]:
# Δημιουργία συνδέσεων και μεγέθους σμηνών κατευθείαν από την edges συνάρτηση-Δυναμικό δίκτυο
mixed_edges_clusters_EPO_App_reg_C=[]
links_EPO_App_reg_C,sizes_EPO_App_reg_C,dates_EPO_App_reg_C=[],[],[]
mixed_edges_clusters_EPO_App_reg_C=creating_edges_fun_dynamical(EPO_App_reg_C_Pub_nbr_Person_id_Appl_id,500)
links_EPO_App_reg_C=mixed_edges_clusters_EPO_App_reg_C[0]
sizes_EPO_App_reg_C=mixed_edges_clusters_EPO_App_reg_C[1]
dates_EPO_App_reg_C=mixed_edges_clusters_EPO_App_reg_C[2]

In [ ]:
for i in range(len(links_EPO_App_reg_C)):
    plt.plot(links_EPO_App_reg_C[i],sizes_EPO_App_reg_C[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_App_reg_C[i],sizes_EPO_App_reg_C[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_App_reg_C[i],sizes_EPO_App_reg_C[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_App_reg_C[i],sizes_EPO_App_reg_C[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_App_reg_C[i],sizes_EPO_App_reg_C[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_App_reg - Full Network - P.C. 'C'",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_App_reg_C_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_App_reg_C)):
    EPO_App_reg_C_DataPlot[i].append([links_EPO_App_reg_C[i],\
                                      sizes_EPO_App_reg_C[i][0],\
                                      sizes_EPO_App_reg_C[i][1],\
                                      sizes_EPO_App_reg_C[i][2],\
                                      sizes_EPO_App_reg_C[i][3],\
                                      sizes_EPO_App_reg_C[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_App_reg_C_DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_App_reg_C_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_App_reg_C_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_App_reg_C)):
    EPO_App_reg_C_Data[i].append([dates_EPO_App_reg_C[i],sizes_EPO_App_reg_C[i]])

with open('FULL_EPO_App_reg_C_Data.txt', 'w') as convert_file:
     for key,value in EPO_App_reg_C_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

## Patents "A": Human necessities

In [ ]:
EPO_App_reg_A_Pub_nbr_Person_id_Appl_id=[] #Aντιστοίχιση για να κρατήσω μόνο τις σειρές με appln_id -> A
EPO_App_reg_A_Pub_nbr_Person_id_Appl_id=appln_id_pub_nbr_person_id_fun(EPO_App_reg_Appln_id,list_Ipc_A)

### Creating Edges for Every Time Window

In [ ]:
# Δημιουργία συνδέσεων και μεγέθους σμηνών κατευθείαν από την edges συνάρτηση-Δυναμικό δίκτυο
mixed_edges_clusters_EPO_App_reg_A=[]
links_EPO_App_reg_A,sizes_EPO_App_reg_A,dates_EPO_App_reg_A=[],[],[]
mixed_edges_clusters_EPO_App_reg_A=creating_edges_fun_dynamical(EPO_App_reg_A_Pub_nbr_Person_id_Appl_id,500)
links_EPO_App_reg_A=mixed_edges_clusters_EPO_App_reg_A[0]
sizes_EPO_App_reg_A=mixed_edges_clusters_EPO_App_reg_A[1]
dates_EPO_App_reg_A=mixed_edges_clusters_EPO_App_reg_A[2]

In [ ]:
for i in range(len(links_EPO_App_reg_A)):
    plt.plot(links_EPO_App_reg_A[i],sizes_EPO_App_reg_A[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_App_reg_A[i],sizes_EPO_App_reg_A[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_App_reg_A[i],sizes_EPO_App_reg_A[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_App_reg_A[i],sizes_EPO_App_reg_A[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_App_reg_A[i],sizes_EPO_App_reg_A[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_App_reg - Full Network - P.C. 'A'",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_App_reg_A_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_App_reg_A)):
    EPO_App_reg_A_DataPlot[i].append([links_EPO_App_reg_A[i],\
                                      sizes_EPO_App_reg_A[i][0],\
                                      sizes_EPO_App_reg_A[i][1],\
                                      sizes_EPO_App_reg_A[i][2],\
                                      sizes_EPO_App_reg_A[i][3],\
                                      sizes_EPO_App_reg_A[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_App_reg_A_DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_App_reg_A_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_App_reg_A_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_App_reg_A)):
    EPO_App_reg_A_Data[i].append([dates_EPO_App_reg_A[i],sizes_EPO_App_reg_A[i]])

with open('FULL_EPO_App_reg_A_Data.txt', 'w') as convert_file:
     for key,value in EPO_App_reg_A_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

## Patents "H": Electricity

In [ ]:
EPO_App_reg_H_Pub_nbr_Person_id_Appl_id=[] #Aντιστοίχιση για να κρατήσω μόνο τις σειρές με appln_id -> H
EPO_App_reg_H_Pub_nbr_Person_id_Appl_id=appln_id_pub_nbr_person_id_fun(EPO_App_reg_Appln_id,list_Ipc_H)

### Creating Edges for Every Time Window

In [ ]:
# Δημιουργία συνδέσεων και μεγέθους σμηνών κατευθείαν από την edges συνάρτηση-Δυναμικό δίκτυο
mixed_edges_clusters_EPO_App_reg_H=[]
links_EPO_App_reg_H,sizes_EPO_App_reg_H,dates_EPO_App_reg_H=[],[],[]
mixed_edges_clusters_EPO_App_reg_H=creating_edges_fun_dynamical(EPO_App_reg_H_Pub_nbr_Person_id_Appl_id,500)
links_EPO_App_reg_H=mixed_edges_clusters_EPO_App_reg_H[0]
sizes_EPO_App_reg_H=mixed_edges_clusters_EPO_App_reg_H[1]
dates_EPO_App_reg_H=mixed_edges_clusters_EPO_App_reg_H[2]

In [ ]:
for i in range(len(links_EPO_App_reg_H)):
    plt.plot(links_EPO_App_reg_H[i],sizes_EPO_App_reg_H[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_App_reg_H[i],sizes_EPO_App_reg_H[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_App_reg_H[i],sizes_EPO_App_reg_H[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_App_reg_H[i],sizes_EPO_App_reg_H[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_App_reg_H[i],sizes_EPO_App_reg_H[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_App_reg - Full Network - P.C. 'H'",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_App_reg_H_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_App_reg_H)):
    EPO_App_reg_H_DataPlot[i].append([links_EPO_App_reg_H[i],\
                                      sizes_EPO_App_reg_H[i][0],\
                                      sizes_EPO_App_reg_H[i][1],\
                                      sizes_EPO_App_reg_H[i][2],\
                                      sizes_EPO_App_reg_H[i][3],\
                                      sizes_EPO_App_reg_H[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_App_reg_H_DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_App_reg_H_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_App_reg_H_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_App_reg_H)):
    EPO_App_reg_H_Data[i].append([dates_EPO_App_reg_H[i],sizes_EPO_App_reg_H[i]])

with open('FULL_EPO_App_reg_H_Data.txt', 'w') as convert_file:
     for key,value in EPO_App_reg_H_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

## Patents "B": Performing Operations; Transporting

In [ ]:
EPO_App_reg_B_Pub_nbr_Person_id_Appl_id=[] #Aντιστοίχιση για να κρατήσω μόνο τις σειρές με appln_id -> B
EPO_App_reg_B_Pub_nbr_Person_id_Appl_id=appln_id_pub_nbr_person_id_fun(EPO_App_reg_Appln_id,list_Ipc_B)

### Creating Edges for Every Time Window

In [ ]:
# Δημιουργία συνδέσεων και μεγέθους σμηνών κατευθείαν από την edges συνάρτηση-Δυναμικό δίκτυο
mixed_edges_clusters_EPO_App_reg_B=[]
links_EPO_App_reg_B,sizes_EPO_App_reg_B,dates_EPO_App_reg_B=[],[],[]
mixed_edges_clusters_EPO_App_reg_B=creating_edges_fun_dynamical(EPO_App_reg_B_Pub_nbr_Person_id_Appl_id,500)
links_EPO_App_reg_B=mixed_edges_clusters_EPO_App_reg_B[0]
sizes_EPO_App_reg_B=mixed_edges_clusters_EPO_App_reg_B[1]
dates_EPO_App_reg_B=mixed_edges_clusters_EPO_App_reg_B[2]

In [ ]:
for i in range(len(links_EPO_App_reg_B)):
    plt.plot(links_EPO_App_reg_B[i],sizes_EPO_App_reg_B[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_App_reg_B[i],sizes_EPO_App_reg_B[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_App_reg_B[i],sizes_EPO_App_reg_B[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_App_reg_B[i],sizes_EPO_App_reg_B[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_App_reg_B[i],sizes_EPO_App_reg_B[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_App_reg - Full Network - P.C. 'B'",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_App_reg_B_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_App_reg_B)):
    EPO_App_reg_B_DataPlot[i].append([links_EPO_App_reg_B[i],\
                                      sizes_EPO_App_reg_B[i][0],\
                                      sizes_EPO_App_reg_B[i][1],\
                                      sizes_EPO_App_reg_B[i][2],\
                                      sizes_EPO_App_reg_B[i][3],\
                                      sizes_EPO_App_reg_B[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_App_reg_B_DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_App_reg_B_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_App_reg_B_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_App_reg_B)):
    EPO_App_reg_B_Data[i].append([dates_EPO_App_reg_B[i],sizes_EPO_App_reg_B[i]])

with open('FULL_EPO_App_reg_B_Data.txt', 'w') as convert_file:
     for key,value in EPO_App_reg_B_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

## Patents "G": Physics

In [ ]:
EPO_App_reg_G_Pub_nbr_Person_id_Appl_id=[] #Aντιστοίχιση για να κρατήσω μόνο τις σειρές με appln_id -> G
EPO_App_reg_G_Pub_nbr_Person_id_Appl_id=appln_id_pub_nbr_person_id_fun(EPO_App_reg_Appln_id,list_Ipc_G)

### Creating Edges for Every Time Window

In [ ]:
# Δημιουργία συνδέσεων και μεγέθους σμηνών κατευθείαν από την edges συνάρτηση-Δυναμικό δίκτυο
mixed_edges_clusters_EPO_App_reg_G=[]
links_EPO_App_reg_G,sizes_EPO_App_reg_G,dates_EPO_App_reg_G=[],[],[]
mixed_edges_clusters_EPO_App_reg_G=creating_edges_fun_dynamical(EPO_App_reg_G_Pub_nbr_Person_id_Appl_id,500)
links_EPO_App_reg_G=mixed_edges_clusters_EPO_App_reg_G[0]
sizes_EPO_App_reg_G=mixed_edges_clusters_EPO_App_reg_G[1]
dates_EPO_App_reg_G=mixed_edges_clusters_EPO_App_reg_G[2]

In [ ]:
for i in range(len(links_EPO_App_reg_G)):
    plt.plot(links_EPO_App_reg_G[i],sizes_EPO_App_reg_G[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_App_reg_G[i],sizes_EPO_App_reg_G[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_App_reg_G[i],sizes_EPO_App_reg_G[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_App_reg_G[i],sizes_EPO_App_reg_G[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_App_reg_G[i],sizes_EPO_App_reg_G[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_App_reg - Full Network - P.C. 'G'",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_App_reg_G_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_App_reg_G)):
    EPO_App_reg_G_DataPlot[i].append([links_EPO_App_reg_G[i],\
                                      sizes_EPO_App_reg_G[i][0],\
                                      sizes_EPO_App_reg_G[i][1],\
                                      sizes_EPO_App_reg_G[i][2],\
                                      sizes_EPO_App_reg_G[i][3],\
                                      sizes_EPO_App_reg_G[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_App_reg_G_DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_App_reg_G_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_App_reg_G_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_App_reg_G)):
    EPO_App_reg_G_Data[i].append([dates_EPO_App_reg_G[i],sizes_EPO_App_reg_G[i]])

with open('FULL_EPO_App_reg_G_Data.txt', 'w') as convert_file:
     for key,value in EPO_App_reg_G_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

# EPO_Inv_reg

## Pub_nbrs 

In [ ]:
EPO_Inv_reg_all=[]
EPO_Inv_reg_all=list(map(list,zip(EPO_Inv_reg['Pub_nbr'],EPO_Inv_reg['Person_id'],EPO_Inv_reg['Appln_id'])))
#EPO_Inv_reg_all.sort(key=takeZeroColumn) # Έχω Sortάρει πάνω με βάση το pub_nbr

In [ ]:
# Τελικές τιμές Pub_nbrs, Person_ids μετά την αντιστοίχιση στις 2 λίστες.
EPO_Inv_reg_Pub_nbr_Person_id=[]
EPO_Inv_reg_Pub_nbr_Person_id=pub_nbr_person_id_fun(EPO_Inv_reg_all,list_EP_Cit_Counts)

In [ ]:
del EP_Cit_Counts
list_EP_Cit_Counts.clear()
test_list_EP_Cit_Counts.clear()
All_Dates.clear()
del EPO_Inv_reg
EPO_Inv_reg_all.clear()

## Creating Edges 

In [ ]:
mixed_edges_clusters_EPO_Inv_reg=[]
links_EPO_Inv_reg,sizes_EPO_Inv_reg,dates_EPO_Inv_reg=[],[],[]
mixed_edges_clusters_EPO_Inv_reg=creating_edges_fun_dynamical(EPO_Inv_reg_Pub_nbr_Person_id,1000)
links_EPO_Inv_reg=mixed_edges_clusters_EPO_Inv_reg[0]
sizes_EPO_Inv_reg=mixed_edges_clusters_EPO_Inv_reg[1]
dates_EPO_Inv_reg=mixed_edges_clusters_EPO_Inv_reg[2]

In [ ]:
for i in range (len(links_EPO_Inv_reg)):
    plt.plot(links_EPO_Inv_reg[i],sizes_EPO_Inv_reg[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_Inv_reg[i],sizes_EPO_Inv_reg[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_Inv_reg[i],sizes_EPO_Inv_reg[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_Inv_reg[i],sizes_EPO_Inv_reg[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_Inv_reg[i],sizes_EPO_Inv_reg[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_Inv_reg - Full Network",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_Inv_reg_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_Inv_reg)):
    EPO_Inv_reg_DataPlot[i].append([links_EPO_Inv_reg[i],\
                                    sizes_EPO_Inv_reg[i][0],\
                                    sizes_EPO_Inv_reg[i][1],\
                                    sizes_EPO_Inv_reg[i][2],\
                                    sizes_EPO_Inv_reg[i][3],\
                                    sizes_EPO_Inv_reg[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_Inv_reg__DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_Inv_reg_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_Inv_reg_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_Inv_reg)):
    EPO_Inv_reg_Data[i].append([dates_EPO_Inv_reg[i],sizes_EPO_Inv_reg[i]])

with open('FULL_EPO_Inv_reg__Data.txt', 'w') as convert_file:
     for key,value in EPO_Inv_reg_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

# EPO_Inv_reg based on pantents' category

In [ ]:
EPO_Inv_reg_Appln_id=[] #Παίρνω το προηγούμενο αρχείο και σορτάρω με βάση το appln_id για αντιστοίχιση παρακάτω
EPO_Inv_reg_Appln_id=EPO_Inv_reg_Pub_nbr_Person_id
EPO_Inv_reg_Appln_id.sort(key=takeSecondColumn)

## Patents "C": Chemistry; Metallurgy

In [ ]:
EPO_Inv_reg_C_Pub_nbr_Person_id_Appl_id=[] #Aντιστοίχιση για να κρατήσω μόνο τις σειρές με appln_id -> C
EPO_Inv_reg_C_Pub_nbr_Person_id_Appl_id=appln_id_pub_nbr_person_id_fun(EPO_Inv_reg_Appln_id,list_Ipc_C)

### Creating Edges for Every Time Window

In [ ]:
# Δημιουργία συνδέσεων και μεγέθους σμηνών κατευθείαν από την edges συνάρτηση-Δυναμικό δίκτυο
mixed_edges_clusters_EPO_Inv_reg_C=[]
links_EPO_Inv_reg_C,sizes_EPO_Inv_reg_C,dates_EPO_Inv_reg_C=[],[],[]
mixed_edges_clusters_EPO_Inv_reg_C=creating_edges_fun_dynamical(EPO_Inv_reg_C_Pub_nbr_Person_id_Appl_id,1000)
links_EPO_Inv_reg_C=mixed_edges_clusters_EPO_Inv_reg_C[0]
sizes_EPO_Inv_reg_C=mixed_edges_clusters_EPO_Inv_reg_C[1]
dates_EPO_Inv_reg_C=mixed_edges_clusters_EPO_Inv_reg_C[2]

In [ ]:
for i in range(len(links_EPO_Inv_reg_C)):
    plt.plot(links_EPO_Inv_reg_C[i],sizes_EPO_Inv_reg_C[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_Inv_reg_C[i],sizes_EPO_Inv_reg_C[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_Inv_reg_C[i],sizes_EPO_Inv_reg_C[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_Inv_reg_C[i],sizes_EPO_Inv_reg_C[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_Inv_reg_C[i],sizes_EPO_Inv_reg_C[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_Inv_reg - Full Network - P.C. 'C'",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_Inv_reg_C_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_Inv_reg_C)):
    EPO_Inv_reg_C_DataPlot[i].append([links_EPO_Inv_reg_C[i],\
                                           sizes_EPO_Inv_reg_C[i][0],\
                                           sizes_EPO_Inv_reg_C[i][1],\
                                           sizes_EPO_Inv_reg_C[i][2],\
                                           sizes_EPO_Inv_reg_C[i][3],\
                                           sizes_EPO_Inv_reg_C[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_Inv_reg_C_DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_Inv_reg_C_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_Inv_reg_C_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_Inv_reg_C)):
    EPO_Inv_reg_C_Data[i].append([dates_EPO_Inv_reg_C[i],sizes_EPO_Inv_reg_C[i]])

with open('FULL_EPO_Inv_reg_C_Data.txt', 'w') as convert_file:
     for key,value in EPO_Inv_reg_C_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

## Patents "A": Human necessities

In [ ]:
EPO_Inv_reg_A_Pub_nbr_Person_id_Appl_id=[] #Aντιστοίχιση για να κρατήσω μόνο τις σειρές με appln_id -> A
EPO_Inv_reg_A_Pub_nbr_Person_id_Appl_id=appln_id_pub_nbr_person_id_fun(EPO_Inv_reg_Appln_id,list_Ipc_A)

### Creating Edges for Every Time Window

In [ ]:
# Δημιουργία συνδέσεων και μεγέθους σμηνών κατευθείαν από την edges συνάρτηση-Δυναμικό δίκτυο
mixed_edges_clusters_EPO_Inv_reg_A=[]
links_EPO_Inv_reg_A,sizes_EPO_Inv_reg_A,dates_EPO_Inv_reg_A=[],[],[]
mixed_edges_clusters_EPO_Inv_reg_A=creating_edges_fun_dynamical(EPO_Inv_reg_A_Pub_nbr_Person_id_Appl_id,1000)
links_EPO_Inv_reg_A=mixed_edges_clusters_EPO_Inv_reg_A[0]
sizes_EPO_Inv_reg_A=mixed_edges_clusters_EPO_Inv_reg_A[1]
dates_EPO_Inv_reg_A=mixed_edges_clusters_EPO_Inv_reg_A[2]

In [ ]:
for i in range(len(links_EPO_Inv_reg_A)):
    plt.plot(links_EPO_Inv_reg_A[i],sizes_EPO_Inv_reg_A[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_Inv_reg_A[i],sizes_EPO_Inv_reg_A[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_Inv_reg_A[i],sizes_EPO_Inv_reg_A[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_Inv_reg_A[i],sizes_EPO_Inv_reg_A[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_Inv_reg_A[i],sizes_EPO_Inv_reg_A[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_Inv_reg - Full Network - P.C. 'A'",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_Inv_reg_A_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_Inv_reg_A)):
    EPO_Inv_reg_A_DataPlot[i].append([links_EPO_Inv_reg_A[i],\
                                      sizes_EPO_Inv_reg_A[i][0],\
                                      sizes_EPO_Inv_reg_A[i][1],\
                                      sizes_EPO_Inv_reg_A[i][2],\
                                      sizes_EPO_Inv_reg_A[i][3],\
                                      sizes_EPO_Inv_reg_A[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_Inv_reg_A_DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_Inv_reg_A_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_Inv_reg_A_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_Inv_reg_A)):
    EPO_Inv_reg_A_Data[i].append([dates_EPO_Inv_reg_A[i],sizes_EPO_Inv_reg_A[i]])

with open('FULL_EPO_Inv_reg_A_Data.txt', 'w') as convert_file:
     for key,value in EPO_Inv_reg_A_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

## Patents "H": Electricity

In [ ]:
EPO_Inv_reg_H_Pub_nbr_Person_id_Appl_id=[] #Aντιστοίχιση για να κρατήσω μόνο τις σειρές με appln_id -> H
EPO_Inv_reg_H_Pub_nbr_Person_id_Appl_id=appln_id_pub_nbr_person_id_fun(EPO_Inv_reg_Appln_id,list_Ipc_H)

### Creating Edges for Every Time Window

In [ ]:
# Δημιουργία συνδέσεων και μεγέθους σμηνών κατευθείαν από την edges συνάρτηση-Δυναμικό δίκτυο
mixed_edges_clusters_EPO_Inv_reg_H=[]
links_EPO_Inv_reg_H,sizes_EPO_Inv_reg_H,dates_EPO_Inv_reg_H=[],[],[]
mixed_edges_clusters_EPO_Inv_reg_H=creating_edges_fun_dynamical(EPO_Inv_reg_H_Pub_nbr_Person_id_Appl_id,1000)
links_EPO_Inv_reg_H=mixed_edges_clusters_EPO_Inv_reg_H[0]
sizes_EPO_Inv_reg_H=mixed_edges_clusters_EPO_Inv_reg_H[1]
dates_EPO_Inv_reg_H=mixed_edges_clusters_EPO_Inv_reg_H[2]

In [ ]:
for i in range(len(links_EPO_Inv_reg_H)):
    plt.plot(links_EPO_Inv_reg_H[i],sizes_EPO_Inv_reg_H[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_Inv_reg_H[i],sizes_EPO_Inv_reg_H[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_Inv_reg_H[i],sizes_EPO_Inv_reg_H[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_Inv_reg_H[i],sizes_EPO_Inv_reg_H[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_Inv_reg_H[i],sizes_EPO_Inv_reg_H[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_Inv_reg - Full Network - P.C. 'H'",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_Inv_reg_H_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_Inv_reg_H)):
    EPO_Inv_reg_H_DataPlot[i].append([links_EPO_Inv_reg_H[i],\
                                      sizes_EPO_Inv_reg_H[i][0],\
                                      sizes_EPO_Inv_reg_H[i][1],\
                                      sizes_EPO_Inv_reg_H[i][2],\
                                      sizes_EPO_Inv_reg_H[i][3],\
                                      sizes_EPO_Inv_reg_H[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_Inv_reg_H_DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_Inv_reg_H_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_Inv_reg_H_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_Inv_reg_H)):
    EPO_Inv_reg_H_Data[i].append([dates_EPO_Inv_reg_H[i],sizes_EPO_Inv_reg_H[i]])

with open('FULL_EPO_Inv_reg_H_Data.txt', 'w') as convert_file:
     for key,value in EPO_Inv_reg_H_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

## Patents "B": Performing Operations; Transporting

In [ ]:
EPO_Inv_reg_B_Pub_nbr_Person_id_Appl_id=[] #Aντιστοίχιση για να κρατήσω μόνο τις σειρές με appln_id -> B
EPO_Inv_reg_B_Pub_nbr_Person_id_Appl_id=appln_id_pub_nbr_person_id_fun(EPO_Inv_reg_Appln_id,list_Ipc_B)

### Creating Edges for Every Time Window

In [ ]:
# Δημιουργία συνδέσεων και μεγέθους σμηνών κατευθείαν από την edges συνάρτηση-Δυναμικό δίκτυο
mixed_edges_clusters_EPO_Inv_reg_B=[]
links_EPO_Inv_reg_B,sizes_EPO_Inv_reg_B,dates_EPO_Inv_reg_B=[],[],[]
mixed_edges_clusters_EPO_Inv_reg_B=creating_edges_fun_dynamical(EPO_Inv_reg_B_Pub_nbr_Person_id_Appl_id,1000)
links_EPO_Inv_reg_B=mixed_edges_clusters_EPO_Inv_reg_B[0]
sizes_EPO_Inv_reg_B=mixed_edges_clusters_EPO_Inv_reg_B[1]
dates_EPO_Inv_reg_B=mixed_edges_clusters_EPO_Inv_reg_B[2]

In [ ]:
for i in range(len(links_EPO_Inv_reg_B)):
    plt.plot(links_EPO_Inv_reg_B[i],sizes_EPO_Inv_reg_B[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_Inv_reg_B[i],sizes_EPO_Inv_reg_B[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_Inv_reg_B[i],sizes_EPO_Inv_reg_B[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_Inv_reg_B[i],sizes_EPO_Inv_reg_B[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_Inv_reg_B[i],sizes_EPO_Inv_reg_B[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_Inv_reg - Full Network - P.C. 'B'",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_Inv_reg_B_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_Inv_reg_B)):
    EPO_Inv_reg_B_DataPlot[i].append([links_EPO_Inv_reg_B[i],\
                                      sizes_EPO_Inv_reg_B[i][0],\
                                      sizes_EPO_Inv_reg_B[i][1],\
                                      sizes_EPO_Inv_reg_B[i][2],\
                                      sizes_EPO_Inv_reg_B[i][3],\
                                      sizes_EPO_Inv_reg_B[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_Inv_reg_B_DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_Inv_reg_B_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_Inv_reg_B_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_Inv_reg_B)):
    EPO_Inv_reg_B_Data[i].append([dates_EPO_Inv_reg_B[i],sizes_EPO_Inv_reg_B[i]])

with open('FULL_EPO_Inv_reg_B_Data.txt', 'w') as convert_file:
     for key,value in EPO_Inv_reg_B_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

## Patents "G": Physics

In [ ]:
EPO_Inv_reg_G_Pub_nbr_Person_id_Appl_id=[] #Aντιστοίχιση για να κρατήσω μόνο τις σειρές με appln_id -> G
EPO_Inv_reg_G_Pub_nbr_Person_id_Appl_id=appln_id_pub_nbr_person_id_fun(EPO_Inv_reg_Appln_id,list_Ipc_G)

### Creating Edges for Every Time Window

In [ ]:
# Δημιουργία συνδέσεων και μεγέθους σμηνών κατευθείαν από την edges συνάρτηση-Δυναμικό δίκτυο
mixed_edges_clusters_EPO_Inv_reg_G=[]
links_EPO_Inv_reg_G,sizes_EPO_Inv_reg_G,dates_EPO_Inv_reg_G=[],[],[]
mixed_edges_clusters_EPO_Inv_reg_G=creating_edges_fun_dynamical(EPO_Inv_reg_G_Pub_nbr_Person_id_Appl_id,1000)
links_EPO_Inv_reg_G=mixed_edges_clusters_EPO_Inv_reg_G[0]
sizes_EPO_Inv_reg_G=mixed_edges_clusters_EPO_Inv_reg_G[1]
dates_EPO_Inv_reg_G=mixed_edges_clusters_EPO_Inv_reg_G[2]

In [ ]:
for i in range(len(links_EPO_Inv_reg_G)):
    plt.plot(links_EPO_Inv_reg_G[i],sizes_EPO_Inv_reg_G[i][0],'^',ms=4,markeredgewidth=0.5,mec='r',mfc='none')
    plt.plot(links_EPO_Inv_reg_G[i],sizes_EPO_Inv_reg_G[i][1],'*',ms=4,markeredgewidth=0.5,mec='g',mfc='none')
    plt.plot(links_EPO_Inv_reg_G[i],sizes_EPO_Inv_reg_G[i][2],'<',ms=4,markeredgewidth=0.5,mec='b',mfc='none')
    plt.plot(links_EPO_Inv_reg_G[i],sizes_EPO_Inv_reg_G[i][3],'>',ms=4,markeredgewidth=0.5,mec='c',mfc='none')
    plt.plot(links_EPO_Inv_reg_G[i],sizes_EPO_Inv_reg_G[i][4],'s',ms=4,markeredgewidth=0.5,mec='y',mfc='none')
plt.legend(('1st cluster','2nd cluster','3rd cluster','4th cluster','5th cluster'),\
           shadow=True, title="Legend", fancybox=True,prop={'size': 11})
plt.locator_params(axis='both',nbins=5)
x_values = plt.gca().get_xticks()
plt.gca().set_xticklabels(['{:,.1f}'.format(x/1000) for x in x_values])
y_values = plt.gca().get_yticks()
plt.gca().set_yticklabels(['{:,.1f}'.format(y/1000) for y in y_values])
plt.xticks(size=15)
plt.yticks(size=15)
plt.xlabel("Number of links (x$10^3$)",size=18)
plt.ylabel("Cluster Size (x$10^3$)",size=18)
plt.title("EPO_Inv_reg - Full Network - P.C. 'G'",size=18)
plt.show()

In [ ]:
# EXPORTING DATAPLOT
EPO_Inv_reg_G_DataPlot=collections.defaultdict(list)
for i in range(len(sizes_EPO_Inv_reg_G)):
    EPO_Inv_reg_G_DataPlot[i].append([links_EPO_Inv_reg_G[i],\
                                      sizes_EPO_Inv_reg_G[i][0],\
                                      sizes_EPO_Inv_reg_G[i][1],\
                                      sizes_EPO_Inv_reg_G[i][2],\
                                      sizes_EPO_Inv_reg_G[i][3],\
                                      sizes_EPO_Inv_reg_G[i][4]]) # einai dict. Gi ayto [i] kai append

with open('FULL_EPO_Inv_reg_G_DataPlot.txt', 'w') as convert_file:
    for key,value in EPO_Inv_reg_G_DataPlot.items():
        for x,y,z,w,q,t in value:
            convert_file.write('%s|%s|%s|%s|%s|%s\n'%(x,y,z,w,q,t))

In [ ]:
EPO_Inv_reg_G_Data=collections.defaultdict(list)
for i in range(len(dates_EPO_Inv_reg_G)):
    EPO_Inv_reg_G_Data[i].append([dates_EPO_Inv_reg_G[i],sizes_EPO_Inv_reg_G[i]])

with open('FULL_EPO_Inv_reg_G.txt', 'w') as convert_file:
     for key,value in EPO_Inv_reg_G_Data.items():
        convert_file.write('%s:%s\n'%(key,value))

In [ ]:
import pyttsx3

engine = pyttsx3.init()
engine.say('i am done')
engine.runAndWait()